In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataset = np.load("proyecto_training_data.npy")

#### 2. Usando sclicing con NumPy separar los datos en 2 datasets: entrenamiento(80 %) y validación y pruebas(20 %).

In [ ]:
# Obtenemos un entero que nos indica en que posición del array podemos obtener el 80%
sliceIndex = int(dataset.shape[0] * 0.8)

# Modifica el orden de las filas
np.random.shuffle(dataset)

# Seleccionamos el 80% para el entrenamiento
dataset_entrenamiento = dataset[:sliceIndex]

# Seleccionamos el 20% para la validación
dataset_validacion = dataset[sliceIndex:]

#### 3. Análisis exploratorio de datos: Para cada variable en el dataset calcular((usando numpy o pandas):

In [ ]:
columnas = ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']

dfEntrenamiento = pd.DataFrame(dataset_entrenamiento, columns=columnas)

columnas = ['Media', 'Minimo', 'Maximo', 'ValorPeak', 'DesviacionEstandar']

dfAnalisis = pd.DataFrame(columns=columnas)

dfAnalisis['Media'] = dfEntrenamiento.mean()
dfAnalisis['Minimo'] = dfEntrenamiento.min()
dfAnalisis['Maximo'] = dfEntrenamiento.max()
dfAnalisis['ValorPeak'] = dfAnalisis['Maximo'] - dfAnalisis['Minimo']
dfAnalisis['DesviacionEstandar'] = dfEntrenamiento.std()

In [ ]:
dfAnalisis.head()

#### 4. Para cada variable en el dataset usar seaborn para graficar un histograma de la variable.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
for col in dfEntrenamiento.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(dfEntrenamiento[col], kde=True)
    plt.title(f'Histograma de {col}', fontsize=16)
    
    plt.xlabel(col, fontsize=14)
    plt.ylabel('Frecuencia', fontsize=14)

    plt.axvline(dfAnalisis.loc[col, 'Media'], color='red', linestyle='--', label='Media')
    plt.axvline(dfAnalisis.loc[col, 'Maximo'], color='green', linestyle='--', label='Máximo')
    plt.axvline(dfAnalisis.loc[col, 'Minimo'], color='blue', linestyle='--', label='Mínimo')
    
    plt.legend()
    plt.show()

#### 5. Para cada variable independiente x:

##### Calcular el coeficiente de correlación entre x y y.

In [ ]:
dfCorrelacionVariables = dfEntrenamiento.corr()

In [ ]:
dfCorrelacionVariables.head()

##### Graficar x vs y(scatterplot) usando matplotlib.
##### Colocar el coeficiente de correlación y colocarlo como parte del título de la gráfica.

In [ ]:
# SalePrice es la variable Y, por lo que se excluye de las X.
variablesX = ['OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']
variableY = 'SalePrice'

for variableX in variablesX:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=variableX, y=variableY, data=dfEntrenamiento)
    coeficiente = dfCorrelacionVariables.loc[[variableX], [variableY]].iloc[0,0]
    plt.title(f"{variableX} - {variableY} - {coeficiente}", fontsize=16)
    plt.xlabel(variableX, fontsize=14)
    plt.ylabel(variableY, fontsize=14)
    plt.grid(True)
    plt.show()

##### Basado en la gráfica y el coeficiente de correlación de cada par x,y elegir las 2 variables con más potencial predictivo es decir las 2 variables que presentan mayor correlación entre dicha variable y la variable dependiente.

##### Segun los diagramas y el dataframe de correlacion de variables, las variables que presentan una mayor correlacion son:
##### 1. X (OverallQual) => Y (SalePrice)
##### 2. X (1stFlrSF) => Y (SalePrice)

In [ ]:
dfCorrelacionVariables.loc[['OverallQual', '1stFlrSF'], ['SalePrice']]

#### 6. Crear una función para entrenar un modelo de regresión lineal de una variable y = β0 + β1 ∗ x.
##### La función recibe como argumentos:
##### 6.1 Vector con la variable independiente x,
##### 6.2 Vector con la variable dependiente y,
##### 6.3 un entero epochs que indica por cuantas iteraciones entrenar el modelo.
##### 6.4 un entero imprimir error cada, que nos indica cada cuantas iteraciones queremos imprimir a través de print: el número de iteración, el error del modelo en esa iteración, si imprimir error cada = 10, se despliega en pantalla el error en las iteraciones: 10,20,30,40,50.
##### 6.5 escalar α(learning rate): es usado como parte de la expresión matemática para actualizar en cada iteración los parámetros del modelo.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
class Regresion:
    #def __init__(self):

    def calcular_gradiente(self, x, y, epochs, imprimir_error, learning_rate = 0.0001):
        
        # Crear una matriz de 2 columnas, la primera columna corresponde al vector de datos x 
        # y la segunda columna de la matriz para todas las filas es igual a 1.
        observaciones = np.column_stack((x, np.ones_like(x)))
        
        # Inicializar los parámetros del modelo en un vector β0 y β1, esto es equivalente a empezar el proceso con una recta inicial 
        # la cual en cada iteración actualizaremos hasta encontrar una que aproxime de buena manera los datos x, y.
        parametros = np.array([0.01, 0.1])
        
        n = len(x)
        historial_parametros = {}
        historial_error = {}
        
        # iterar las epocas
        for i in range(epochs):
            
            # Calcular y (prediccioón o estimación) para todas las observaciones de manera simultánea(vectorizada) 
            # utilizando el modelo correspondiente a la iteración(es decir , los valores de β0 y β1,): 
            # esto produce un vector y con el mismo número de elementos que y.
            # y = β0 + β1 * x.
            #y_estimacion =  parametros[0] + (parametros[1] * x)
            y_estimacion = np.dot(observaciones, parametros)
            
            # Calcular el error o costo usando
            error_actual = np.mean((y - y_estimacion) ** 2)
            
            # Calcular el gradiente del error respecto de cada parámetro
            # Realizar un solo cálculo cuyo resultado es un vector
            gradientes = np.dot(observaciones.T, y_estimacion - y) / n
            parametros -= learning_rate * gradientes 

            historial_parametros[i] = parametros
            historial_error[i] = error_actual
            
            i += 1
            mostrar_error = i % imprimir_error

            # Muestra el error cada intervalo especificado en el parametro imprimir_error
            if mostrar_error == 0:
                print(f"Iteration No. {i}: Error {error_actual}, Parametros (0,1) {parametros[0]}, {parametros[1]}")
                #print(f"Iteration No. {i}: Error {error_actual}")
                
        return historial_parametros, historial_error
    
    def grafica_historial_error(self, historial_errores):
        iteraciones = list(historial_errores.keys())
        errores = list(historial_errores.values())
        plt.figure(figsize=(10, 6))
        plt.plot(iteraciones, errores, marker='o', color='blue')
        plt.title('Historial de error por iteración')
        plt.xlabel('Número de iteración')
        plt.ylabel('Error')
        #plt.grid(True)
        plt.show()
            
    def graficar_historial_modelo(self, historial_modelos, n, x, y):
        # Graficar el modelo para cada iteración a graficar
        obs = np.column_stack((x, np.ones_like(x)))
        
        plt.figure(figsize=(10, 6))
        plt.scatter(x, y, color='blue', label='Entrenamiento manual')
        
        for iteracion, parametros in historial_modelos.items():
            if iteracion % n == 0:
                y_pred = np.dot(obs, parametros) #parametros[0] + parametros[1] * x
                plt.plot(x, y_pred, color='red', label=f'Iteración {iteracion}')
        
        plt.title('Evolución del modelo entrenado en el tiempo')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.legend()
        plt.grid(True)
        plt.show()
                
                
    def sklearn_calcular_gradiente(self, X, Y, x_label):
        historial_parametros = {}
        historial_error = {}
        
        linear_regresion = LinearRegression()
        linear_regresion.fit(X.reshape(-1, 1), Y)

        y_pred = linear_regresion.predict(X.reshape(-1, 1))

        error = np.mean((Y - y_pred)**2)

        plt.figure(figsize=(10, 6))
        plt.scatter(X, Y, color='blue', label='Entrenamiento sklearn')
        plt.plot(X, y_pred, color='red', label='Predicción')
        plt.title(f'{x_label} vs SalePrice')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.legend()
        plt.grid(True)
        plt.show()

#### Entrenando la variable 1 donde Y es SalePrice y X es OverallQual

In [1017]:
regresion = Regresion()
X1 = dataset_entrenamiento[:,1]
Y = dataset_entrenamiento[:,0]

In [1016]:
historial_parametros, historial_error = regresion.calcular_gradiente(X1, Y, 8000, 4000, 0.01)

Iteration No. 4000: Error 2340942103.7913876, Parametros (0,1) 43613.02549882527, -84803.90996934645
Iteration No. 8000: Error 2328503556.4496894, Parametros (0,1) 45769.061833581014, -98596.86025508425


In [ ]:
regresion.grafica_historial_error(historial_error)
regresion.graficar_historial_modelo(historial_parametros, 4000, X1, Y)

#### Entrenando la variable 2 donde Y es SalePrice y X es 1stFlrSF

In [1031]:
regresion = Regresion()
X2 = dataset_entrenamiento[:,2]
Y = dataset_entrenamiento[:,0]

X2.reshape(-1, 1)

array([[ 691.],
       [1779.],
       [ 884.],
       ...,
       [ 774.],
       [ 976.],
       [1422.]])

In [1027]:
historial_parametros, historial_error = regresion.calcular_gradiente(X2, Y, 13, 10, 0.0000001)

Iteration No. 10: Error 5947341611.810787, Parametros (0,1) 122.30194927038133, 0.19830584844761476


In [ ]:
regresion.grafica_historial_error(historial_error)
regresion.graficar_historial_modelo(historial_parametros, 4, X2, Y)

#### Para cada una de las variables utilizar Scikit-Learn

In [ ]:
regresion.sklearn_calcular_gradiente(X1, Y, 'OverallQual')
regresion.sklearn_calcular_gradiente(X2, Y, '1stFlrSF')